## Resources

- Berserk documentation: https://berserk.readthedocs.io/en/master/index.html

- Lichess API documentation: https://lichess.org/api

- Lichess Database: https://database.lichess.org/

## Imports

In [1]:
import dotenv
import requests
import os
import berserk
import random
import pandas as pd
import numpy as np
from datetime import date
from tqdm import tqdm

## Load API token and usernames

The current username list is retrieved from the game data found on database.lichess.org

In [2]:
# get token:
api_token = os.environ.get('LICHESS_API_TOKEN')

# import usernames as list:
with open("../data/usernames.txt", "r") as file:
    users = file.read().split(",")

## Get a sample of users, start a session

In [5]:
users_sample = random.sample(users, k=15)

# opening a session to access lichess data:
session = berserk.TokenSession(api_token)
client = berserk.Client(session)

## Request data from the API

In [6]:
# rating dataframes:
rapid_ratings = pd.DataFrame(columns=['username', "year", 	"month", 	"day", 	"rating"])
puzzle_ratings = pd.DataFrame(columns=['username', "year", 	"month", 	"day", 	"rating"])

# rapid games dataframe:
rapid_games = pd.DataFrame()

for user in tqdm(users_sample):
    try:
      user_history = client.users.get_rating_history(user)
      if len(user_history[2]["points"]) > 50 and len(user_history[2]["points"]) < 500: # only extract data if this user has more than 50 rated rapid games.

        # rating data:
        # rapid ratings
        user_rapid_ratings = pd.DataFrame(user_history[2]["points"])
        user_rapid_ratings.insert(0, "username", user)
        user_rapid_ratings['month'] += 1  # because months in lichess API start at 0 we have to increment by 1
        rapid_ratings = pd.concat([rapid_ratings, user_rapid_ratings])

        # puzzle ratings
        user_puzzle_ratings = pd.DataFrame(user_history[13]["points"])
        user_puzzle_ratings.insert(0, "username", user)
        user_puzzle_ratings['month'] += 1  # because months in lichess API start at 0 we have to increment by 1
        puzzle_ratings = pd.concat([puzzle_ratings, user_puzzle_ratings])

        # rapid games information:
        user_games = pd.DataFrame(list(client.games.export_by_player(user, evals=True, clocks=True, opening=True, perf_type="rapid")))
        user_games.insert(0, "username", user)
        rapid_games = pd.concat([rapid_games, user_games])
    except KeyError:
      pass
    except berserk.exceptions.ResponseError:
      pass

100%|██████████| 15/15 [00:50<00:00,  3.36s/it]


## Formatting Data

### Rapid Ratings

In [7]:
rapid_ratings["date"] = pd.to_datetime(rapid_ratings[rapid_ratings.columns[1:4]])
rapid_ratings = rapid_ratings.drop(labels=["year","month","day"], axis=1)
rapid_ratings

,username,rating,date
0,ichbintoll,1795,2017-12-02
1,ichbintoll,1803,2017-12-05
2,ichbintoll,1808,2017-12-06
3,ichbintoll,1799,2017-12-07
4,ichbintoll,1816,2017-12-09
...,...,...,...
282,ichbintoll,1746,2024-04-06
283,ichbintoll,1725,2024-04-19
284,ichbintoll,1710,2024-04-27
285,ichbintoll,1755,2024-07-06


### Puzzle Ratings

In [136]:
puzzle_ratings["date"] = pd.to_datetime(puzzle_ratings[puzzle_ratings.columns[1:4]])
puzzle_ratings = puzzle_ratings.drop(labels=["year","month","day"], axis=1)
puzzle_ratings

,username,rating,date
0,volamcaothu,1384,2019-07-14
1,volamcaothu,1464,2019-08-31
2,volamcaothu,1414,2019-09-14
3,volamcaothu,1544,2019-09-16
4,volamcaothu,1638,2019-09-17
...,...,...,...
117,vodovorot1951,2335,2023-09-18
118,vodovorot1951,2389,2023-09-25
119,vodovorot1951,2444,2023-10-02
120,vodovorot1951,2337,2023-10-03


### Rapid Games

In [8]:
# drop games with nonstandard starting position:
rapid_games = rapid_games[rapid_games["variant"]=="standard"]

# drop games that were not started:
rapid_games = rapid_games[rapid_games.status!="noStart"]

# reset indices:
rapid_games = rapid_games.reset_index(drop=True)


# change id to link for the game:
rapid_games.loc[:,"id"] = "https://lichess.org/" + rapid_games.loc[:,"id"]


# extract white and black columns from players column:
players = pd.DataFrame(rapid_games["players"].to_list())
players = players.applymap(lambda x: x.get("user", {}).get("name"))
rapid_games = pd.concat([rapid_games, players], axis=1)


# extract opening names:
rapid_games["opening"] = rapid_games.apply(lambda x: x["opening"], axis=1).apply(lambda x: x.get("name") if not pd.isna(x) else x)


# extract evaluations:
def extract_eval(x):
    if isinstance(x, list):
        return [item['eval'] for item in x if isinstance(item, dict) and 'eval' in item]
    else:
        return []
rapid_games['analysis'] = rapid_games['analysis'].apply(extract_eval)


# add outcome of the game (win, draw, loss) from the perspective of the username:
conditions = [
    rapid_games["winner"].isna(),
    (rapid_games["winner"]=="white") & (rapid_games["white"]==rapid_games["username"]),
    (rapid_games["winner"]=="black") & (rapid_games["black"]==rapid_games["username"]),
]
choices = [
    "draw",
    "win",
    "win"
]
rapid_games["outcome"] = np.select(conditions, choices, default="loss")


# drop unnecessary columns:
rapid_games = rapid_games.drop(labels=["variant","perf","players","swiss","initialFen","tournament"], axis=1)

C:\Users\phili\AppData\Local\Temp\ipykernel_19648\1502889337.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  players = players.applymap(lambda x: x.get("user", {}).get("name"))


KeyError: "['swiss'] not found in axis"

In [9]:
rapid_games

,username,id,rated,variant,speed,perf,createdAt,lastMoveAt,status,source,...,opening,moves,clocks,analysis,clock,tournament,initialFen,white,black,outcome
0,ichbintoll,https://lichess.org/HT3o5v7m,True,standard,rapid,rapid,2024-08-27 12:35:28.452000+00:00,2024-08-27 13:02:20.349000+00:00,mate,pool,...,Hungarian Opening: Sicilian Invitation,g3 c5 Bg2 Nc6 e3 d5 d4 cxd4 exd4 Nf6 Ne2 Bf5 c...,"[60003, 60003, 60371, 60451, 60707, 60891, 611...","[8, 14, 14, 15, -20, -16, -19, -24, -21, -14, ...","{'initial': 600, 'increment': 5, 'totalTime': ...",NaN,NaN,ichbintoll,bk642022,loss
1,ichbintoll,https://lichess.org/gVsJD2Iv,True,standard,rapid,rapid,2024-07-06 12:39:03.473000+00:00,2024-07-06 13:00:28.076000+00:00,outoftime,pool,...,Hungarian Opening: Catalan Formation,g3 d5 Bg2 e6 e3 Nf6 Ne2 Be7 O-O O-O d4 c5 c4 N...,"[60003, 60003, 60499, 58483, 60171, 57651, 599...","[12, 12, -17, 15, -4, -5, -13, -2, -10, -10, -...","{'initial': 600, 'increment': 5, 'totalTime': ...",NaN,NaN,ichbintoll,wellking,win
2,ichbintoll,https://lichess.org/DM0teNxB,True,standard,rapid,rapid,2024-07-06 11:06:43.493000+00:00,2024-07-06 11:36:46.192000+00:00,resign,pool,...,"Sicilian Defense: Sozin Attack, Flank Variation",e4 c5 Nf3 d6 d4 cxd4 Nxd4 Nf6 Nc3 a6 Bc4 b5 Bb...,"[60003, 60003, 60155, 60371, 60283, 60875, 606...","[15, 25, 17, 26, 26, 28, 18, 24, 32, 22, 6, 42...","{'initial': 600, 'increment': 5, 'totalTime': ...",NaN,NaN,l4sanl,ichbintoll,win
3,ichbintoll,https://lichess.org/jruTlWvm,True,standard,rapid,rapid,2024-04-27 07:03:46.666000+00:00,2024-04-27 07:27:06.019000+00:00,outoftime,pool,...,"Trompowsky Attack: Classical Defense, Big Cent...",d4 Nf6 Bg5 e6 e4 Be7 Nd2 h6 Bh4 d5 e5 Nfd7 Bxe...,"[60003, 60003, 60203, 60283, 59715, 60219, 586...","[12, 22, 0, 12, 6, 60, 36, 51, 29, 28, 29, 28,...","{'initial': 600, 'increment': 5, 'totalTime': ...",NaN,NaN,KmArkouneMlindeAli,ichbintoll,loss
4,ichbintoll,https://lichess.org/s9wIuJmw,True,standard,rapid,rapid,2024-04-19 16:20:47.244000+00:00,2024-04-19 16:42:45.268000+00:00,resign,pool,...,Hungarian Opening,g3 e5 Bg2 Nf6 e3 h6 Ne2 Bc5 d4 exd4 exd4 Bb6 O...,"[60003, 60003, 60299, 59987, 60715, 60275, 610...",[],"{'initial': 600, 'increment': 5, 'totalTime': ...",NaN,NaN,ichbintoll,papik555,loss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,ichbintoll,https://lichess.org/ksn77TJA,True,standard,rapid,rapid,2015-08-31 19:46:35.764000+00:00,2015-08-31 19:58:38.640000+00:00,resign,arena,...,"English Opening: Anglo-Indian Defense, Hedgeho...",c4 Nf6 Nc3 e6 Qc2 d5 e3 Bb4 b3 c5 Bb2 Nc6 a3 B...,NaN,[],"{'initial': 600, 'increment': 0, 'totalTime': ...",rKCdRW6K,NaN,V13,ichbintoll,loss
1429,ichbintoll,https://lichess.org/IKwlTrmW,True,standard,rapid,rapid,2015-08-31 19:28:35+00:00,2015-08-31 19:45:54.814000+00:00,resign,arena,...,Nimzo-Indian Defense,d4 Nf6 c4 e6 Nc3 Bb4 Bd2 d5 a3 Bxc3 Bxc3 c5 e3...,NaN,"[13, 14, 12, 11, 15, 25, 20, 20, -1, 0, -3, 65...","{'initial': 600, 'increment': 0, 'totalTime': ...",rKCdRW6K,NaN,T_king,ichbintoll,loss
1430,ichbintoll,https://lichess.org/dJmHWrMD,True,standard,rapid,rapid,2015-08-31 19:17:53.955000+00:00,2015-08-31 19:27:37.027000+00:00,mate,arena,...,Nimzowitsch Defense: Franco-Nimzowitsch Variation,e4 e6 Nf3 Nc6 Bb5 a6 Ba4 b5 Bb3 Nf6 Nc3 b4 Ne2...,NaN,"[20, 13, 13, 43, 29, 30, 20, 6, 14, 27, -50, -...","{'initial': 600, 'increment': 0, 'totalTime': ...",rKCdRW6K,NaN,ichbintoll,mazdaedraki,loss
1431,ichbintoll,https://lichess.org/bnTkikYM,True,standard,rapid,rapid,2015-08-31 19:10:46.605000+00:00,2015-08-31 19:17:46.940000+00:00,mate,arena,...,Scandinavian Defense: Mieses-Kotroc Variation,e4 d5 exd5 Qxd5 d4 Bf5 Nf3 Nf6 Bd3 Bxd3 Qxd3 e...,NaN,[],"{'initial': 600, 'increment': 0, 'totalTime': ...",rKCdRW6K,NaN,ichbintoll,reti70,win


## Save as .csv

In [140]:
puzzle_ratings.to_csv("../data/puzzle_ratings_Dec15.csv", index=False)
rapid_ratings.to_csv("../data/rapid_ratings_Dec15.csv", index=False)
rapid_games.to_csv("../data/rapid_games_Dec15.csv", index=False)